In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [ ]:
url = 'https://cobslab.com/wp-content/uploads/2022/02/ai-009-1.jpg'
!wget {url}

In [ ]:
img = cv2.imread('/kaggle/working/ai-009-1.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

# 정규화와 표준화

## 정규화

In [ ]:
normalized_img = img / 255.0
print(img[0][0], normalized_img[0][0])

In [ ]:
33plt.imshow(normalized_img)

## 표준화

In [ ]:
mean = np.mean(img, axis=(0, 1))

std = np.std(img, axis=(0, 1))

print(mean, std)

In [ ]:
# 

In [ ]:
plt.imshow(standardized_img)

# 필터링

## Salt and Pepper Noise

In [ ]:
def generate_salt_noise(image):
    num_salt = np.ceil(0.05 * image.size)
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
    salted_image = image.copy()
    salted_image[coords[0], coords[1]] = 255
    return salted_image

In [ ]:
def generate_pepper_noise(image):
    num_pepper = np.ceil(0.05 * image.size)
    coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
    peppered_image = image.copy()
    peppered_image[coords[0], coords[1]] = 0
    return peppered_image

In [ ]:
!wget https://raw.githubusercontent.com/Cobslab/imageBible/main/image/like_lenna.png

In [ ]:
lenna_path = '/kaggle/working/like_lenna.png'

In [ ]:
lenna_image = cv2.imread(lenna_path, cv2.IMREAD_GRAYSCALE)

In [ ]:
#
#
filtered_lenna = cv2.medianBlur(peppered_lenna, 5)


fig, axes = plt.subplots(1, 4, figsize=(20, 6))

import matplotlib.pyplot as plt

axes[0].imshow(lenna_image, cmap='gray')
axes[0].set_title('Original Lenna Image')
axes[0].axis('off')

axes[1].imshow(salted_lenna, cmap='gray')
axes[1].set_title('Salted Lenna Image')
axes[1].axis('off')

axes[2].imshow(peppered_lenna, cmap='gray')
axes[2].set_title('Salted & Peppered Lenna')
axes[2].axis('off')

axes[3].imshow(filtered_lenna, cmap='gray')
axes[3].set_title('Median Filtered Lenna')
axes[3].axis('off')

plt.tight_layout()
plt.show()

# 가우시안 필터링

In [ ]:
image = cv2.imread(lenna_path, cv2.IMREAD_GRAYSCALE)

mean = 0
sigma = 1
gaussian_noise = np.random.normal(mean, sigma, image.shape).astype('uint8')
noisy_image = cv2.add(image, gaussian_noise)

plt.imshow(noisy_image, cmap='gray')
plt.title('Noisy Image')
plt.axis('off')
plt.show()

In [ ]:
#
denoised_images = []

for sigma in sigma_values:
    denoised = cv2.GaussianBlur(noisy_image, (0, 0), sigma)
    denoised_images.append(denoised)

fig, axes = plt.subplots(1, 4, figsize=(20, 10))

axes[0].imshow(noisy_image, cmap='gray')
axes[0].set_title('Noisy Image')
axes[0].axis('off')

for ax, img, sigma in zip(axes[1:], denoised_images, sigma_values):
    ax.imshow(img, cmap='gray')
    ax.set_title(f'Denoised (σ={sigma})')
    ax.axis('off')

plt.tight_layout()
plt.show()

# 변환

## 원근 변환

In [ ]:
!wget https://raw.githubusercontent.com/Lilcob/test_colab/main/perspective_test.jpg

In [ ]:
# 이미지 로드
image_path = '/kaggle/working/perspective_test.jpg'
new_source_image = cv2.imread(image_path)

# 지정한 꼭지점 좌표 (좌표 순서 변경)
ordered_corners = np.array([[57, 630], [936, 330], [1404, 792], [550, 1431]], dtype='float32')

# 너비와 높이 계산
ordered_width = int(max(np.linalg.norm(ordered_corners[0] - ordered_corners[1]),
                        np.linalg.norm(ordered_corners[2] - ordered_corners[3])))
ordered_height = int(max(np.linalg.norm(ordered_corners[0] - ordered_corners[3]),
                         np.linalg.norm(ordered_corners[1] - ordered_corners[2])))

# 변환이 될 꼭지점 좌표 지정
ordered_rect_corners = np.array([[0, 0], [ordered_width, 0], [ordered_width, ordered_height], [0, ordered_height]], dtype='float32')

# 호모그래피 행렬 계산
ordered_scan_matrix = cv2.getPerspectiveTransform(ordered_corners, ordered_rect_corners)

# 원근 변환 다시 적용
ordered_scanned_image = cv2.warpPerspective(new_source_image, ordered_scan_matrix, (ordered_width, ordered_height))

# 스캔된 이미지 다시 출력
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("New Source Image")
plt.imshow(cv2.cvtColor(new_source_image, cv2.COLOR_BGR2RGB))
plt.subplot(1, 2, 2)
plt.title("Ordered Scanned Image")
plt.imshow(cv2.cvtColor(ordered_scanned_image, cv2.COLOR_BGR2RGB))
plt.show()

# 이미지 피라미드

In [ ]:
image = cv2.imread(lenna_path, cv2.IMREAD_COLOR)

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def gaussian_pyramid(image, levels):
    pyramid = [image]
    for i in range(levels-1):
        image = cv2.pyrDown(image)
        pyramid.append(image)
    return pyramid

levels = 5
pyramid = gaussian_pyramid(image_rgb, levels)

# 가우시안 피라미드를 시각화
fig, axes = plt.subplots(1, levels, figsize=(20, 8))
for i, ax in enumerate(axes):
    ax.imshow(pyramid[i])
    ax.axis('off')
    ax.set_title(f'Level {i+1}')

plt.tight_layout()
plt.show()

# 경계 검출(Edge Detection)

## Canny Edge Detection

In [ ]:
image = cv2.imread(lenna_path, cv2.IMREAD_COLOR)

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 가우시안 블러 적용
blurred_image = cv2.GaussianBlur(image_rgb, (5, 5), 1.4)

# 2. 캐니 에지 검출
canny_edges = cv2.Canny(blurred_image, threshold1=50, threshold2=150)

#3. 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(image_rgb)
axes[0].axis('off')
axes[0].set_title('Original Image')
axes[1].imshow(blurred_image)
axes[1].axis('off')
axes[1].set_title('Gaussian Blurred Image')
axes[2].imshow(canny_edges, cmap='gray')
axes[2].axis('off')
axes[2].set_title('Canny Edge Detection')

In [ ]:
# 다양한 임계값 조합으로 캐니 에지 검출 수행
thresholds = [(10, 50), (50, 100), (100, 150), (150, 200)]

canny_results = []

for threshold1, threshold2 in thresholds:
    canny_image = cv2.Canny(blurred_image, threshold1=threshold1, threshold2=threshold2)
    canny_results.append(canny_image)

# 시각화
fig, axes = plt.subplots(1, len(thresholds), figsize=(20, 5))

for i, ax in enumerate(axes):
    ax.imshow(canny_results[i], cmap='gray')
    ax.axis('off')
    ax.set_title(f'threshold1={thresholds[i][0]}, threshold2={thresholds[i][1]}')

## Prewitt Edge Detection



In [ ]:
image = cv2.imread(lenna_path, cv2.IMREAD_GRAYSCALE)

# 프리윗 커널 정의
kx = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
ky = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])

# 프리윗 커널 적용
gx = cv2.filter2D(image, -1, kx)
gy = cv2.filter2D(image, -1, ky)

# 결과 이미지
prewitt_result = cv2.addWeighted(gx, 0.5, gy, 0.5, 0)

# 결과 출력
plt.imshow(prewitt_result, cmap='gray')
plt.axis('off')
plt.title('Prewitt Edge Detection')
plt.show()

## Sobel Edge Detection

In [ ]:
# 소벨 커널 적용
gx_sobel = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
gy_sobel = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

# 결과 이미지
sobel_result = cv2.addWeighted(gx_sobel, 0.5, gy_sobel, 0.5, 0)

# 결과 출력
plt.imshow(sobel_result, cmap='gray')
plt.axis('off')
plt.title('Sobel Edge Detection')
plt.show()

## Hough Transform
허프 변환(Hough Transform)은 이미지에서 선을 검출하는 데 사용됩니다. `cv2.HoughLinesP` 함수는 일반적인 허프 변환과 달리 선의 시작점과 끝점을 반환하므로, 특정 길이 이상의 선을 찾는 데 유용합니다.

In [ ]:
!wget -O football_stadium.jpg "https://www.arsenal.com/sites/default/files/styles/link_image_extra_large/public/images/GettyImages-884411976.jpg?auto=webp&itok=w7-4LWKS"

football_stadium_path = '/kaggle/working/football_stadium.jpg'

In [ ]:
image = cv2.imread(football_stadium_path, cv2.IMREAD_COLOR)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

plt.imshow(gray, cmap='gray')
plt.axis('off')
plt.title('Original Image')
plt.show()

# Canny 에지 검출
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# 확장된 허프 변환을 사용하여 선 검출
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)

# 검출된 선을 원본 이미지 위에 그리기
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 선을 녹색으로 그리기

# 결과 이미지 시각화
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Detected Lines using HoughLinesP')
plt.show()

# 이미지 마스킹
`cv2.inRange()` 함수는 OpenCV에서 특정 색상 범위 내에 있는 픽셀을 찾아서 이진화 마스크를 생성하는 데 사용됩니다. 이 함수는 입력 이미지에서 지정된 범위 내의 색상을 가진 픽셀을 찾고, 그 범위에 속하는 픽셀은 255(흰색), 나머지 픽셀은 0(검정색)으로 설정된 마스크 이미지를 반환합니다.

In [ ]:
image = cv2.imread(football_stadium_path, cv2.IMREAD_COLOR)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Original Image')
plt.show()

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 붉은색 범위
lower_red = np.array([-30, 0, 0])
upper_red = np.array([30, 255, 255])

# 붉은 범위에 해당하는 마스크 생성
mask = cv2.inRange(hsv, lower_red, upper_red)

# 원본 이미지에서 붉은 부분만 추출
# 첫 번째 image와, 두 번째 image+mask 사이의 교집합
result = cv2.bitwise_and(image, image, mask=mask)

plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Masked Image')
plt.show()

# 히스토그램 분석
OpenCV에서 히스토그램 연산을 하는 이유는 이미지의 픽셀 값 분포를 분석하고, 이를 통해 이미지의 특성을 파악하거나, 이미지 처리 작업에서 필요한 정보를 얻기 위해서입니다. 히스토그램은 이미지에서 각 픽셀 값이 얼마나 자주 나타나는지를 그래프로 나타낸 것으로, 이미지의 밝기, 대비, 색상 분포 등을 이해하는 데 유용합니다.

In [ ]:
image = cv2.imread(lenna_path, cv2.IMREAD_COLOR)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Original Image')
plt.show()

# BGR 채널로 분리
channels = cv2.split(image) # 이미지 채널 별 분리
colors = ('b', 'g', 'r')
plt.figure(figsize=(10, 6))

# 각 채널에 대해 히스토그램 계산 및 시각화
for (channel, color) in zip(channels, colors):
    # calcHist(채널, 인덱스, 마스크, 히스토그램 빈 수, 픽셀 범위)
    hist = cv2.calcHist([channel], [0], None, [256], [0, 256])
    plt.plot(hist, color=color)

# 그래프 제목과 레이블 설정
plt.title('Color Histogram for BGR Image')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# 동일한 이미지에 대하여 가우시안 필터를 적용한 뒤 히스토그램 비교해보기

# Thresholding
주로 그레이스케일 이미지에서 특정 임계값(threshold)을 기준으로 픽셀을 이진화(binary)하는 과정입니다. 이 과정은 이미지를 흑백으로 변환하거나, 특정 영역을 분리하는 데 사용됩니다. 예를 들어, 문서 스캔 이미지에서 텍스트와 배경을 분리하거나, 객체를 추출할 때 사용됩니다.

## Binary Thresholding

In [ ]:
img = cv2.imread(lenna_path, cv2.IMREAD_GRAYSCALE)

ret, th1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

plt.imshow(th1, cmap='gray')
plt.axis('off')
plt.title('Global Thresholding (v = 127)')
plt.show()

## Adaptive Thresholding

Adaptive Thresholding(적응형 임계값 처리)은 이미지에서 조명 조건이 균일하지 않을 때 효과적으로 사용할 수 있는 기법입니다. 일반적인 Thresholding 방법이 이미지 전체에 동일한 임계값을 적용하는 반면, Adaptive Thresholding은 이미지의 작은 영역마다 서로 다른 임계값을 적용하여 이진화 처리를 수행합니다.

In [ ]:
img = cv2.imread(lenna_path, cv2.IMREAD_GRAYSCALE)


img = cv2.medianBlur(img, 5)
 
ret, th1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

th2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, \
            cv2.THRESH_BINARY, 11, 2)

th3 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, \
            cv2.THRESH_BINARY, 11, 2)
 
titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']

images = [img, th1, th2, th3]
 
for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

## Otsu's Thresholding

Otsu's Thresholding은 이미지 이진화에 사용되는 자동 임계값 결정 기법입니다. Otsu의 방법은 이미지의 히스토그램을 분석하여 클래스 간의 분산이 최대가 되는 임계값을 자동으로 선택합니다. 이 방법은 이미지의 밝기 분포에 따라 최적의 임계값을 계산하므로, 사용자가 임계값을 수동으로 설정할 필요 없이 효과적으로 이진화를 수행할 수 있습니다.

In [ ]:
img = cv2.imread(lenna_path, cv2.IMREAD_GRAYSCALE)
 
# global thresholding
ret1,th1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
 
# Otsu's thresholding
ret2,th2 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
 
# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(img, (5,5), 0)
ret3,th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
 
# plot all the images and their histograms
images = [img, 0, th1,
          img, 0, th2,
          blur, 0, th3]
titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)',
          'Original Noisy Image','Histogram',"Otsu's Thresholding",
          'Gaussian filtered Image','Histogram',"Otsu's + Gaussian"]
 
for i in range(3):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
plt.show()

# Image Segmentation

## Watershed Algorithm

이미지 처리에서 객체의 경계를 감지하고 분할하는 데 사용되는 강력한 기법입니다. 이 알고리즘의 이름은 물이 지형을 따라 흐르는 방식을 시뮬레이션하는 방식에서 유래했습니다. 이미지를 지형으로 간주하고, 물이 낮은 지점부터 흘러가는 것을 시뮬레이션하여 서로 다른 객체를 분리합니다.

In [ ]:
coin_url = "https://docs.opencv.org/4.x/water_coins.jpg"
!wget {coin_url} -O coin.jpg

coin_path = '/kaggle/working/coin.jpg'

In [ ]:
image = cv2.imread(coin_path, cv2.IMREAD_COLOR)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Original Image')
plt.show()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

plt.imshow(thresh, cmap='gray')
plt.axis('off')
plt.title('Otsu Thresholding')
plt.show()

# 미니 프로젝트 - Lane Detection


- 제시된 동영상 파일에서 차선을 검출하여 gif로 변환하는 코드를 작성해주세요.

- 아래 지침에 따라 코드를 설계해주세요.

1. 동영상 파일을 연 후, 각 프레임마다 특정 색상을 검출합니다.

2. 색상을 검출하기 위하여 이미지를 HSV 형식으로 변환합니다.

3. 흰 부분의 값을 HSV로 정의하고, 이를 에지 검출 알고리즘으로 분리합니다.

4. 분리된 선을 바탕으로 허프 변환을 가합니다.

5. 처리된 각 프레임을 RGB로 변환합니다.

6. imageio.mimsave 메서드로 여러 프레임을 gif로 변환합니다.